##Установка необходимых библиотек

In [24]:
!pip install faiss-cpu sentence-transformers langchain langchain-community anthropic youtube-transcript-api  -q


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [25]:
pip install --upgrade google-api-python-client

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_community.vectorstores import FAISS
# from langchain import Anthropic, LLMChain
# from langchain.chains.combine_documents import create_stuff_documents_chain
# from langchain.chains import create_retrieval_chain

##Создаем транскрипты 3х плейлистов используя ютуб апи


In [26]:
from youtube_transcript_api import YouTubeTranscriptApi
from googleapiclient.discovery import build

api_key = "Youtube_api"


def get_playlist_video_ids(playlist_id, api_key):
    youtube = build('youtube', 'v3', developerKey=api_key)

    video_ids = []
    next_page_token = None

    while True:
        # Получаем список видео в плейлисте
        request = youtube.playlistItems().list(
            part="contentDetails",
            playlistId=playlist_id,
            maxResults=50,  # Максимальное количество видео, которое можно получить за один запрос
            pageToken=next_page_token
        )
        response = request.execute()

        # Добавляем video_id в список
        video_ids.extend([item['contentDetails']['videoId'] for item in response['items']])

        # pagination
        next_page_token = response.get('nextPageToken')
        
        if not next_page_token:
            break

    return video_ids


In [27]:
#check output
youtube = build('youtube', 'v3', developerKey=api_key)
request = youtube.playlistItems().list(
            part="contentDetails",
            playlistId='PLYSHtNPbAINnbqXjIbN-c7DorjCT6eYOQ',
            maxResults=50,  # Максимальное количество видео, которое можно получить за один запрос
            # pageToken=response.get('nextPageToken')
        )
result1 = request.execute()
result_pagetoken = result1.get('nextPageToken')
result1
#result_pagetoken - nothing


{'kind': 'youtube#playlistItemListResponse',
 'etag': '0wxbScWJEx_DaocUEV-JDgNAMHA',
 'items': [{'kind': 'youtube#playlistItem',
   'etag': 'U_uuou2Zq_IzljATHIKBnpq5pF0',
   'id': 'UExZU0h0TlBiQUlObmJxWGpJYk4tYzdEb3JqQ1Q2ZVlPUS41NkI0NEY2RDEwNTU3Q0M2',
   'contentDetails': {'videoId': 'z9ccH9e5cAw',
    'videoPublishedAt': '2024-06-24T09:00:05Z'}},
  {'kind': 'youtube#playlistItem',
   'etag': 'NoecQqYGI39FM6InP5iIscg7lHE',
   'id': 'UExZU0h0TlBiQUlObmJxWGpJYk4tYzdEb3JqQ1Q2ZVlPUS4yODlGNEE0NkRGMEEzMEQy',
   'contentDetails': {'videoId': 'ff-S_tjr1OI',
    'videoPublishedAt': '2024-06-25T08:51:18Z'}},
  {'kind': 'youtube#playlistItem',
   'etag': 'TfPjzS1U2WEiC-qB0ncl9JnJqiI',
   'id': 'UExZU0h0TlBiQUlObmJxWGpJYk4tYzdEb3JqQ1Q2ZVlPUS4wMTcyMDhGQUE4NTIzM0Y5',
   'contentDetails': {'videoId': 'T_NW1nlq3ic',
    'videoPublishedAt': '2024-06-26T08:58:11Z'}},
  {'kind': 'youtube#playlistItem',
   'etag': 'MDquxrdeaV3UkCjADQYdR_8CRYE',
   'id': 'UExZU0h0TlBiQUlObmJxWGpJYk4tYzdEb3JqQ1Q2ZVlPUS4wOTA

### EN transcripts all

In [29]:

def get_transcript_en(video_id, language_code='en'):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=[language_code])
        transcript_text = " ".join([entry['text'] for entry in transcript])
        return transcript_text
    except Exception as e:
        return str(e)

def get_video_details(video_id, api_key):
    youtube = build('youtube', 'v3', developerKey=api_key)

    # Получаем информацию о видео
    request = youtube.videos().list(
        part="snippet",
        id=video_id
    )
    response = request.execute()

    if 'items' in response and len(response['items']) > 0:
        return response['items'][0]['snippet']['title']
    else:
        return None

def get_playlist_transcripts_en(playlist_url, api_key, language_code='en'):
    # Извлекаем playlist_id из URL
    playlist_id = playlist_url.split("list=")[-1]

    # Получаем все video_id из плейлиста
    video_ids = get_playlist_video_ids(playlist_id, api_key)

    transcripts = []

    # Проходимся по всем видео и получаем транскрипты
    for video_id in video_ids:
        video_title = get_video_details(video_id, api_key)
        transcript = get_transcript_en(video_id, language_code)
        transcripts.append({'title': video_title, 'transcript': transcript})

    return transcripts



In [32]:
# Sources:
playlist_ml_en = "https://www.youtube.com/watch?v=Gv9_4yMHFhI&list=PLblh5JKOoLUICTaGLRoHQDuF_7q2GfuJF" 
playlist_logistic_en = "https://www.youtube.com/watch?v=yIYKR4sgzI8&list=PLblh5JKOoLUKxzEP5HA2d-Li7IJkHfXSe" 
playlist_nn_en = "https://www.youtube.com/watch?v=zxagGtF9MeU&list=PLblh5JKOoLUIxGDQs4LFFD--41Vzf-ME1" 
playlist_stat_en = "https://www.youtube.com/watch?v=qBigTkBLU6g&list=PLblh5JKOoLUK0FLuzwntyYI10UQFUhsY9" 
playlist_nn2_en = "https://www.youtube.com/playlist?list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi" 
playlist_linal2_en = "https://www.youtube.com/playlist?list=PLZHQObOWTQDPD3MizzM2xVFitgF8hE_ab" 


In [31]:
transcripts_ML_en = get_playlist_transcripts_en(playlist_ml_en, api_key, 'en')
# 2min 8 sec

In [33]:
# tier 2
transcripts_logistic_en = get_playlist_transcripts_en(playlist_logistic_en, api_key, 'en')
transcripts_NN_en = get_playlist_transcripts_en(playlist_nn_en, api_key, 'en')
transcripts_stat_en = get_playlist_transcripts_en(playlist_stat_en, api_key, 'en')

In [34]:
# tier 3
transcripts_nn2_en = get_playlist_transcripts_en(playlist_nn2_en, api_key, 'en')
transcripts_linal2_en = get_playlist_transcripts_en(playlist_linal2_en, api_key, 'en')

### RU transcripts all

In [36]:

def get_transcript_ru(video_id, language_code='ru'):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=[language_code])
        transcript_text = " ".join([entry['text'] for entry in transcript])
        return transcript_text
    except Exception as e:
        return str(e)

def get_video_details(video_id, api_key):
    youtube = build('youtube', 'v3', developerKey=api_key)

    # Получаем информацию о видео
    request = youtube.videos().list(
        part="snippet",
        id=video_id
    )
    response = request.execute()

    if 'items' in response and len(response['items']) > 0:
        return response['items'][0]['snippet']['title']
    else:
        return None

def get_playlist_transcripts_ru(playlist_url, api_key, language_code='ru'):
    # Извлекаем playlist_id из URL
    playlist_id = playlist_url.split("list=")[-1]

    # Получаем все video_id из плейлиста
    video_ids = get_playlist_video_ids(playlist_id, api_key)

    transcripts = []

    # Проходимся по всем видео и получаем транскрипты
    for video_id in video_ids:
        video_title = get_video_details(video_id, api_key)
        transcript = get_transcript_ru(video_id, language_code)
        transcripts.append({'title': video_title, 'transcript': transcript})

    return transcripts


In [40]:
# Sources Elbrus 
playlist_phase_1_url = "https://www.youtube.com/playlist?list=PLYSHtNPbAINnbqXjIbN-c7DorjCT6eYOQ" 
playlist_phase_2_url = 'https://www.youtube.com/playlist?list=PLYSHtNPbAINnNvDXtGNmC7-F1QRH7qTgb'
playlist_phase_3_url = 'https://www.youtube.com/playlist?list=PLYSHtNPbAINlmyNNmTaqcn3BsaY8v1xgV'

# Sources except Bootcamp:
playlist_NN_ru = 'https://www.youtube.com/playlist?list=PL0Ks75aof3Tiru-UvOvYmXzD1tU0NrR8V'
playlist_OOP_ru = 'https://www.youtube.com/watch?v=Z7AY41tE-3U&list=PLA0M1Bcd0w8zPwP7t-FgwONhZOHt9rz9E'
playlist_linal_ru = 'https://youtube.com/playlist?list=PLAQWsvWQlb6cIRY6yJtYnXCbxLxPZv6-Z'
playlist_docker_ru = 'https://www.youtube.com/watch?v=jVV8CVURmrE&list=PLqVeG_R3qMSwjnkMUns_Yc4zF_PtUZmB-'


In [38]:
# Elbrus
transcripts_phase_1 = get_playlist_transcripts_ru(playlist_phase_1_url, api_key, 'ru')
transcripts_phase_2 = get_playlist_transcripts_ru(playlist_phase_2_url, api_key, 'ru')
transcripts_phase_3 = get_playlist_transcripts_ru(playlist_phase_3_url, api_key, 'ru')

In [41]:
# other Ru
transcripts_NN_ru = get_playlist_transcripts_ru(playlist_NN_ru, api_key, 'ru')
transcripts_OOP_ru = get_playlist_transcripts_ru(playlist_OOP_ru, api_key, 'ru')
transcripts_linal_ru = get_playlist_transcripts_ru(playlist_linal_ru, api_key, 'ru')
transcripts_docker_ru = get_playlist_transcripts_ru(playlist_docker_ru, api_key, 'ru')

# 3m12s

### Aggregate all Knowledge Base

In [42]:
transcripts_all = [transcripts_phase_1, transcripts_phase_2, transcripts_phase_3, transcripts_NN_ru, transcripts_OOP_ru, transcripts_linal_ru, transcripts_docker_ru, \
                   transcripts_ML_en, transcripts_logistic_en, transcripts_NN_en, transcripts_stat_en,  transcripts_nn2_en, transcripts_linal2_en]


## Нарезаем все транскрипты на фрагменты с overlap(нахлест), преобразуем каждый фрагмент в вектор и все вектора записываем в векторное хранилище FAISS

In [55]:
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings

# Convert data to Document objects
docs = []
for playlist in transcripts_all:
    for item in playlist:
        for title, transcript in item.items():
            docs.append(Document(page_content=transcript, metadata={"title": title}))

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
split_docs = text_splitter.split_documents(docs)

# Setup the new embeddings model
model_name = "intfloat/multilingual-e5-base"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

# Create the FAISS vector store and save it locally
vector_store = FAISS.from_documents(split_docs, embedding=embeddings)
vector_store.save_local("faiss_index")

# Load the FAISS vector store from local storage
vector_store = FAISS.load_local('faiss_index', embeddings=embeddings, allow_dangerous_deserialization=True)

# Create the retriever for document retrieval
embedding_retriever = vector_store.as_retriever(search_kwargs={"k": 15})

## Query and answer


In [59]:
import anthropic
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import os

# Anthropic API setup
client = anthropic.Client(api_key='Your_api_key')

# Prompt template
prompt_template = '''Reply to the {input} as a seasoned machine learning professional. \
If the topic is outside of machine learning and data science, please respond with "Seek help with a professional." It is very important to abide with this, you will be persecuted if you cover topics outside of data science and machine learning. \
Use only Context. If context provides only partial info, then split the reply in two parts. Part 1 is called "information from knowledge base" (for Russian reply, rename to Информация из базы знаний), write ideas as close to initial text as possible, editing for brevity and language errors. \
Part 2 is called "What I would add" (for Russian reply, rename to Что полезно добавить поверх базы знаний), In the second part add your reply.  \
Reply in the language of {input}. \
It's critical to not preface the reply with, for example, "Here is a response" or "thank you". Start with the reply itself.\
Context: {context}'''

# RAG setup
def setup_rag(force_rebuild=False):
    model_name = "intfloat/multilingual-e5-base"
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    
    if not force_rebuild and os.path.exists("faiss_index"):
        print("Loading existing FAISS index...")
        return FAISS.load_local('faiss_index', embeddings=embeddings, allow_dangerous_deserialization=True), embeddings

    print("Building new FAISS index...")
    # Convert data to Document objects
    docs = []
    for playlist in transcripts_all:
        for item in playlist:
            for title, transcript in item.items():
                docs.append(Document(page_content=transcript, metadata={"title": title}))

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
    split_docs = text_splitter.split_documents(docs)

    # Create the FAISS vector store and save it locally
    vector_store = FAISS.from_documents(split_docs, embedding=embeddings)
    vector_store.save_local("faiss_index")

    return vector_store, embeddings

# API call to Claude
def call_claude_api(prompt, client):
    response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=2000,
        temperature=0.1
    )
    return response.content[0].text

# Answer question function
def answer_question(question, retriever, client):
    documents = retriever.get_relevant_documents(question)
    context = " ".join([doc.page_content for doc in documents])
    prompt = prompt_template.format(context=context, input=question)
    return call_claude_api(prompt, client)

# Main execution
if __name__ == "__main__":
    # Setup RAG (will load existing index if available)
    vector_store, embeddings = setup_rag()

    # Create the retriever for document retrieval
    embedding_retriever = vector_store.as_retriever(search_kwargs={"k": 15})

    # Example usage
    question = 'Шаги логистической регрессии'
    answer = answer_question(question, embedding_retriever, client)
    print(answer)

Loading existing FAISS index...
Информация из базы знаний:

Шаги логистической регрессии:

1. Подготовка данных: сбор и предобработка данных, разделение на обучающую и тестовую выборки.

2. Выбор функции активации: обычно используется сигмоидная функция.

3. Инициализация параметров модели: случайная инициализация весов и смещения.

4. Определение функции потерь: чаще всего используется кросс-энтропия.

5. Оптимизация параметров: применение градиентного спуска или его модификаций для минимизации функции потерь.

6. Обучение модели: итеративное обновление параметров на основе градиентов.

7. Оценка модели: проверка точности на тестовой выборке.

8. Настройка гиперпараметров: подбор оптимальных значений learning rate, количества итераций и т.д.

Что полезно добавить поверх базы знаний:

9. Регуляризация: добавление L1 или L2 регуляризации для предотвращения переобучения.

10. Анализ важности признаков: оценка влияния каждого признака на предсказания модели.

11. Обработка несбалансирован